# Import pandas and ast

In [1]:
import pandas as pd
import ast

# Loading in data and parsing entries with lists of objects

In [2]:
main_member_df = pd.read_csv('../congress_members_2024.csv')

# Creating index for members later on
main_member_df['member_id'] = main_member_df.index


# Parsing string representation back into list
main_member_df['all_industries_parsed'] = main_member_df['all_industries'].apply(ast.literal_eval)
main_member_df['all_contributors_parsed'] = main_member_df['all_contributors'].apply(ast.literal_eval)
main_member_df['all_funding_types_parsed'] = main_member_df['funding_type'].apply(ast.literal_eval)
main_member_df['total_vs_avg_raised_parsed'] = main_member_df['total_vs_avg_raised'].apply(ast.literal_eval)


# Fields for this are
# contribution_type, contribution_amount, contribution_percentage


# year, total_raised_by_congressman, average_raised
main_member_df['total_vs_avg_raised_parsed']


0      [{'year': 2014, 'total_raised_by_congressman':...
1      [{'year': 1996, 'total_raised_by_congressman':...
2      [{'year': 2012, 'total_raised_by_congressman':...
3      [{'year': 2022, 'total_raised_by_congressman':...
4      [{'year': 2012, 'total_raised_by_congressman':...
                             ...                        
534    [{'year': 2010, 'total_raised_by_congressman':...
535    [{'year': 1990, 'total_raised_by_congressman':...
536    [{'year': 2022, 'total_raised_by_congressman':...
537    [{'year': 2010, 'total_raised_by_congressman':...
538    [{'year': 2014, 'total_raised_by_congressman':...
Name: total_vs_avg_raised_parsed, Length: 539, dtype: object

# Helper Functions to convert string to integers

In [3]:
def convert_money_to_int(str_money):
    if str_money[0] == '-':
        str_money = str_money[1:]
        return -int(str_money.strip('$').replace(',', ''))


    return int(str_money.strip('$').replace(',', ''))


def convert_fields_to_int(df):

    print(type(df))

    df['total'] = df['total'].apply(convert_money_to_int)
    df['individuals'] = df['individuals'].apply(convert_money_to_int)
    df['pacs'] = df['pacs'].apply(convert_money_to_int)

    return df


# Create industries dataframe to link back to main dataframe

In [4]:
industries_list = []
industries_df = None

for idx, row in main_member_df.iterrows():
    member_id = row['member_id']
    for industry in row['all_industries_parsed']:
        industries_list.append(
            {
                'member_id': member_id,
                'industry': str(industry['industry']),
                'total': str(industry['total']),
                'individuals': str(industry['individuals']),
                'pacs': str(industry['pacs'])

            }
        )

    industries_df = pd.DataFrame(industries_list)

industries_df = convert_fields_to_int(industries_df)
industries_df.head()



<class 'pandas.core.frame.DataFrame'>


,member_id,industry,total,individuals,pacs
0,0,Commercial Banks,48245,3745,44500
1,0,Public Sector Unions,44000,1000,43000
2,0,Building Trade Unions,40500,0,40500
3,0,Insurance,36980,980,36000
4,0,Health Professionals,32720,7220,25500


# Create contributors dataframe for members

In [5]:
contributors_list = []
contributors_df = None

for idx, row in main_member_df.iterrows():
    member_id = row['member_id']
    for contributor in row['all_contributors_parsed']:
        contributors_list.append(
            {
                'member_id': member_id,
                'contributor': str(contributor['contributor']),
                'total': str(contributor['total']),
                'individuals': str(contributor['individuals']),
                'pacs': str(contributor['pacs'])

            }
        )
    contributors_df = pd.DataFrame(contributors_list)


contributors_df = convert_fields_to_int(contributors_df)
contributors_df.head()


<class 'pandas.core.frame.DataFrame'>


,member_id,contributor,total,individuals,pacs
0,0,Duke Energy,10350,350,10000
1,0,American Crystal Sugar,10000,0,10000
2,0,American Federation of State/Cnty/Munic Employees,10000,0,10000
3,0,American Federation of Teachers,10000,0,10000
4,0,Blue Cross & Blue Shield Assn,10000,0,10000


# Create all funding types for individual dataframe

In [6]:
all_funding_list = []
all_funding_df = None

for idx, row in main_member_df.iterrows():
    member_id = row['member_id']
    for funding in row['all_funding_types_parsed']:
        # contribution_type, contribution_amount, contribution_percentage
        all_funding_list.append(
            {
                'member_id': member_id,
                'contribution_type': str(funding['contribution_type']),
                'contribution_amount': funding['contribution_amount'],
                'contribution_percent': funding['contribution_percent'],

            }
        )
    all_funding_df = pd.DataFrame(all_funding_list)


all_funding_df.head(20)




,member_id,contribution_type,contribution_amount,contribution_percent
0,0,pac_contributions,516244,72
1,0,large_individual_contributions,164760,23
2,0,small_individual_contributions_200,34494,4
3,0,candidate_selffinancing,0,0
4,0,other,-560,0
5,1,pac_contributions,795500,54
6,1,large_individual_contributions,611924,41
7,1,other,39797,2
8,1,small_individual_contributions_200,23428,1
9,1,candidate_selffinancing,0,0


# Get total vs average raised by congress member dataframe

In [7]:
# year, total_raised_by_congressman, average_raised

total_vs_avg_list = []
total_vs_avg_df = None

for idx, row in main_member_df.iterrows():
    member_id = row['member_id']
    for entry in row['total_vs_avg_raised_parsed']:
        total_vs_avg_list.append(
            {
                'member_id': member_id,
                'year': entry['year'],
                'total_raised_by_congress_member': entry['total_raised_by_congressman'],
                'average_raised_by_congress_member': entry['average_raised'],

            }
        )
    total_vs_avg_df = pd.DataFrame(total_vs_avg_list)


total_vs_avg_df.head(30)




,member_id,year,total_raised_by_congress_member,average_raised_by_congress_member
0,0,2014,766900.0,1720000
1,0,2016,922090.0,1750000
2,0,2018,589690.0,2190000
3,0,2020,824120.0,2640000
4,0,2022,947660.0,3060000
5,0,2024,714940.0,3310000
6,1,1996,759350.0,747230
7,1,1998,1510000.0,890710
8,1,2000,1640000.0,1010000
9,1,2002,712480.0,980640


In [8]:
columns_to_drop = ['all_industries', 'all_contributors', 'total_vs_avg_raised', 'funding_type', 'all_industries_parsed', 'all_contributors_parsed', 'total_vs_avg_raised_parsed', 'all_funding_types_parsed']

main_member_df.drop(columns = columns_to_drop, inplace = True)


main_member_df.head()


,link,name,state,chamber,party,raised,spent,cash_on_hand,debts,top_industry,top_contributor,top_industry_number,top_contributor_number,first_election,election_type,election_year,member_id
0,https://www.opensecrets.org/members-of-congres...,Alma Adams,North Carolina,House,Democrat,714938,684971,573409,0,Commercial Banks,Duke Energy,"$48,245","$10,350",2014,Next Election,2024.0,0
1,https://www.opensecrets.org/members-of-congres...,Robert B Aderholt,Alabama,House,Republican,1470649,1663198,966896,0,Misc Defense,American Israel Public Affairs Cmte,"$141,034","$34,804",1996,Next Election,2024.0,1
2,https://www.opensecrets.org/members-of-congres...,Pete Aguilar,California,House,Democrat,5808795,4739480,2649071,0,Pro-Israel,American Israel Public Affairs Cmte,"$673,463","$620,898",2014,Next Election,2024.0,2
3,https://www.opensecrets.org/members-of-congres...,Mark Alford,Missouri,House,Republican,1303886,1140087,217408,0,Retired,Plz Aeroscience,"$91,292","$26,400",2022,Next Election,2024.0,3
4,https://www.opensecrets.org/members-of-congres...,Richard W Allen,Georgia,House,Republican,1217539,901827,1174956,1639945,Retired,American Israel Public Affairs Cmte,"$96,386","$25,550",2014,Next Election,2024.0,4


# Party and Ideology Analysis

# Party information

In [9]:
# Finding who raised the most by party
raised_by_party = main_member_df.groupby('party')['raised'].sum()


# How much members for each party
party_counts = main_member_df['party'].value_counts()


# Average spending per party
spending_by_party = main_member_df.groupby('party')[['raised', 'spent']].mean()

# Geographic patterns

In [35]:
# States the raise the most amount of money
main_member_df.groupby('state')['raised'].sum().nlargest(20)


# Top industries for each state
top_industry_per_state = main_member_df.groupby('state')['top_industry'].apply(lambda x: x.value_counts().nlargest(1))

print(top_industry_per_state)


state                                                   
Alabama               Retired                                3
Alaska                Retired                                2
American Samoa        Leadership PACs                        1
Arizona               Retired                                9
Arkansas              Crop Production & Basic Processing     2
California            Retired                               15
Colorado              Retired                                6
Connecticut           Retired                                3
Delaware              Pharmaceuticals/Health Products        1
District of Columbia  Transportation Unions                  1
Florida               Retired                               10
Georgia               Retired                                5
Guam                  Leadership PACs                        1
Hawaii                Retired                                2
Idaho                 Securities & Investment                

# Industry Influence

In [11]:
# Finding what industries contribute the most
top_industries = industries_df.groupby('industry')['total'].sum().nlargest(10)
top_industries

industry
Retired                    586889379
Securities & Investment    166595294
Lawyers/Law Firms          111412208
Republican/Conservative     94353384
Real Estate                 79903156
Democratic/Liberal          62255173
Education                   54710754
Health Professionals        52612941
Leadership PACs             48506379
Pro-Israel                  30066449
Name: total, dtype: int64

# House vs Senate

In [12]:
# Total raised by senate vs total raised by house
total_raised_chamber = main_member_df.groupby('chamber')['raised'].sum()


# Total members for each chamber
total_for_chamber = main_member_df['chamber'].value_counts()


average_raised_by_house_member = total_raised_chamber[0] / total_for_chamber[0]
average_raised_by_senate_member = total_raised_chamber[1] / total_for_chamber[1]




# top industry for senators
top_industries_senate = main_member_df[main_member_df['chamber'] == 'Senate']['top_contributor'].value_counts()

# Top industry for house members
top_industries_house = main_member_df[main_member_df['chamber'] == 'House']['top_contributor'].value_counts()


top_industries_senate


top_contributor
American Israel Public Affairs Cmte         23
Senate Conservatives Fund                    6
Blackstone Group                             5
Apollo Global Management                     5
Pro-Israel America PAC                       3
EMILY's List                                 3
Google Inc                                   3
Stephens Inc                                 2
National Republican Senatorial Cmte          2
JStreetPAC                                   2
Club for Growth                              2
Beasley, Allen et al                         2
Lockheed Martin                              1
University of Washington                     1
ConocoPhillips                               1
Building Bridges Pac                         1
University of Michigan                       1
Bloomberg Lp                                 1
Yale University                              1
steptoe LLP                                  1
Donohoe Companies                           

# Top industries per party per state

In [52]:
main_member_df.head()
top_industry_per_party_per_state = main_member_df.groupby(['state','party'])['top_industry'].apply(lambda x: x.value_counts().nlargest(1))


top_industry_per_party_per_state['Texas']

party                
Democrat    Oil & Gas     4
Republican  Retired      13
Name: top_industry, dtype: int64

# Longest tenured members in congress

In [59]:
earliest_year = main_member_df['first_election'].min()
longest_tenured_member = main_member_df[main_member_df['first_election'] == earliest_year]

longest_tenured_member

,link,name,state,chamber,party,raised,spent,cash_on_hand,debts,top_industry,top_contributor,top_industry_number,top_contributor_number,first_election,election_type,election_year,member_id
193,https://www.opensecrets.org/members-of-congres...,Chuck Grassley,Iowa,Senate,Republican,597606,706128,332900,0,Retired,National Republican Senatorial Cmte,"$1,665,154","$52,200",1980,Next Election,2028.0,193
408,https://www.opensecrets.org/members-of-congres...,Hal Rogers,Kentucky,House,Republican,1052276,793990,971251,0,Lobbyists,Cornerstone Government Affairs,"$97,711","$29,000",1980,Next Election,2024.0,408
457,https://www.opensecrets.org/members-of-congres...,Chris Smith,New Jersey,House,Republican,838536,700904,364065,0,Retired,American Israel Public Affairs Cmte,"$73,282","$27,500",1980,Next Election,2024.0,457


# Average increase by member per year

In [80]:

# compute mean year-to-year change per member and broadcast to all their rows
total_vs_avg_df['avg_growth_for_member'] = (
    total_vs_avg_df.groupby('member_id')['total_raised_by_congress_member']
      .transform(lambda s: s.diff().mean())
)


largest_average_growth = total_vs_avg_df['avg_growth_for_member'].max()
member_with_largest_growth = total_vs_avg_df[total_vs_avg_df['avg_growth_for_member'] == largest_average_growth]

member_with_largest_growth




,member_id,year,total_raised_by_congress_member,average_raised_by_congress_member,avg_growth_for_member
37,5,2018,5850000.0,2190000,28910000.0
38,5,2020,5670000.0,2640000,28910000.0
39,5,2022,3390000.0,3060000,28910000.0
40,5,2024,92580000.0,3310000,28910000.0
